In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/server

/content/drive/MyDrive/server


In [ ]:
!pip install flask pyngrok torch torchvision torchaudio diffusers transformers accelerate pillow "imageio[ffmpeg]"


In [ ]:
!pip install flask-cors

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2vaKMRs8ypMaSp3NuQCiaA0x3ZF_2RnEkC3hs1W17vUiqZTbW")

In [ ]:
# pip install -q flask flask-cors pyngrok diffusers transformers accelerate pillow torch torchvision torchaudio
import os, gc, uuid, torch
from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from pyngrok import ngrok
from diffusers import DiffusionPipeline

# Start tunnel BEFORE routes so we can use public_url in responses
public_url = ngrok.connect(5500).public_url
print("Public URL:", public_url)

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": ["http://127.0.0.1:5500","http://localhost:5500"]}})

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.float16 if device=="cuda" else torch.float32
NEG    = "blurry, low-res, distorted, watermark, text, logo"

sdxl = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=dtype
).to(device)

@app.get("/")
def root():
    return "Colab Flask SDXL server is running!"

# Serve images from /content
@app.get("/files/<path:filename>")
def serve_file(filename):
    # now https://<ngrok>/files/<name>.png will read from /content/<name>.png
    return send_from_directory("/content", filename, as_attachment=False)

@app.post("/get-image")
def get_image():
    data   = request.get_json(force=True) or {}
    prompt = (data.get("prompt") or "a cat in space").strip()
    steps  = int(data.get("steps", 26))
    seed   = int(data.get("seed", 123))

    gen = torch.Generator(device=device).manual_seed(seed)
    with torch.inference_mode():
        img = sdxl(
            prompt=prompt, negative_prompt=NEG,
            num_inference_steps=steps, guidance_scale=6.0,
            height=512, width=512, generator=gen,
        ).images[0]

    fname = f"{uuid.uuid4().hex}.png"
    fpath = f"/content/{fname}"    # <-- stays in /content
    img.save(fpath)

    del img, gen; gc.collect()
    if device == "cuda":
        torch.cuda.empty_cache(); torch.cuda.ipc_collect()

    # Return ABSOLUTE URL on your ngrok domain
    return jsonify({"link": f"{public_url}/files/{fname}"})

app.run(port=5500)


Public URL: https://3b256f781b23.ngrok-free.app


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:17:51] "POST /get-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:17:52] "GET /files/206da252a2ab416aa776d82cba4939d1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:28] "OPTIONS /get-image HTTP/1.1" 200 -


  0%|          | 0/26 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:32] "POST /get-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:32] "GET /files/18e945ea1fa74baaa0d0d3b386f82bc1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:44] "OPTIONS /get-image HTTP/1.1" 200 -


  0%|          | 0/26 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:47] "POST /get-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Sep/2025 15:18:48] "GET /files/9cf523c6fa3f4b118b38927bc46200fa.png HTTP/1.1" 200 -
